In [1]:
import cv2
import torch
from ultralytics import YOLO
import time

# 1. Load Pre-trained YOLOv8 Model
model = YOLO("yolov8m.pt")  # Load a small YOLOv8 model

In [2]:


# 2. Load and Process Video Frame
video_path = r'C:\Users\PREDATOR\Downloads\congestion classifier\Untitled video - Made with Clipchamp.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Get the frame rate of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_interval = fps * 1  # 2 seconds interval

# Start frame count
frame_count = 0

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Process one frame every 2 seconds
    if frame_count % frame_interval == 0:
        # Get the frame width and height
        frame_width = frame.shape[1]
        frame_height = frame.shape[0]

        # 3. Define Region of Interest (ROI)
        roi_top_left = (int(frame_width * 0.4), int(frame_height * 0.1))
        roi_bottom_right = (int(frame_width * 0.8), int(frame_height * 0.8))

        # 4. Extract ROI from the Frame
        roi = frame[roi_top_left[1]:roi_bottom_right[1], roi_top_left[0]:roi_bottom_right[0]]

        # 5. Run YOLOv8 on the ROI
        results = model(roi)

        # 6. Count Vehicles (Filtering for vehicle classes only)
        vehicle_count = 0
        for result in results:
            for detection in result.boxes:
                # Check if the detected object's class is in the list of vehicle classes
                if int(detection.cls) in [2, 3, 5, 7]:  # Car, Motorcycle, Bus, Truck
                    vehicle_count += 1

        # 7. Calculate Density (Assume ROI area is 100 square units for this example)
        density = vehicle_count / 100

        # 8. Define Congestion Thresholds (Adjust as needed)
        high_congestion_threshold = 0.2
        moderate_congestion_threshold = 0.1

        # 9. Classify Congestion Level
        if density >= high_congestion_threshold:
            congestion_level = "High Congestion"
        elif density >= moderate_congestion_threshold:
            congestion_level = "Moderate Congestion"
        else:
            congestion_level = "Low Congestion"

        # 10. Display Results (Including Bounding Boxes)
        for detection in result.boxes:
            if int(detection.cls) in [2, 3, 5, 7]:  # Filtering for vehicle classes
                x1, y1, x2, y2 = map(int, detection.xyxy[0])
                # Adjust coordinates to account for ROI offset
                x1 += roi_top_left[0]
                x2 += roi_top_left[0]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw rectangle around ROI (Right half of the frame)
        cv2.rectangle(frame, roi_top_left, roi_bottom_right, (0, 255, 0), 2)
        cv2.putText(frame, f"Density: {density:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Congestion: {congestion_level}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Traffic Congestion Analysis", frame)

    # Increment frame count
    frame_count += 1

    # 11. Exit on Key Press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 640x640 3 persons, 5 cars, 2 motorcycles, 1 bus, 7 trucks, 16.0ms
Speed: 5.0ms preprocess, 16.0ms inference, 109.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 9 cars, 3 buss, 6 trucks, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 11 cars, 1 motorcycle, 2 buss, 4 trucks, 13.1ms
Speed: 2.0ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 9 cars, 2 motorcycles, 2 buss, 8 trucks, 14.0ms
Speed: 2.9ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 10 cars, 2 motorcycles, 3 buss, 5 trucks, 1 cake, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 9 cars, 3 motorcycles, 2 buss, 5 trucks, 14.1ms
Speed: 2.0ms preprocess, 14.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640

KeyboardInterrupt: 

In [33]:
# 2. Load and Process Video Frame
video_path = r'C:\Users\PREDATOR\Downloads\congestion classifier\Untitled video - Made with Clipchamp.mp4' # Replace with your video file path
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Get the frame width and height
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

    # 3. Define Region of Interest (ROI)
    # Adjust the ROI to focus on a specific portion of the right side of the road
    roi_top_left = (int(frame_width * 0.4), int(frame_height * 0.1))  # Start at 50% width and 20% height
    roi_bottom_right = (int(frame_width * 0.8), int(frame_height * 0.8))  # End at full width and 80% height

    # 4. Extract ROI from the Frame
    roi = frame[roi_top_left[1]:roi_bottom_right[1], roi_top_left[0]:roi_bottom_right[0]]

    # 5. Run YOLOv8 on the ROI
    results = model(roi)

    # 6. Count Vehicles
    vehicle_count = 0 
    for result in results:
        for detection in result.boxes.xyxy:
            vehicle_count += 1

    # 7. Calculate Density (Assume ROI area is 100 square units for this example)
    density = vehicle_count / 100

    # 8. Define Congestion Thresholds (Adjust as needed)
    high_congestion_threshold = 0.2
    moderate_congestion_threshold = 0.1

    # 9. Classify Congestion Level
    if density >= high_congestion_threshold:
        congestion_level = "High Congestion"
    elif density >= moderate_congestion_threshold:
        congestion_level = "Moderate Congestion"
    else:
        congestion_level = "Low Congestion"

    # 10. Display Results (Including Bounding Boxes)
    # Draw bounding boxes for detected vehicles
    for result in results:
        for detection in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, detection)
            # Adjust coordinates to account for ROI offset
            x1 += roi_top_left[0]
            x2 += roi_top_left[0]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 

    # Draw rectangle around ROI (Right half of the frame)
    cv2.rectangle(frame, roi_top_left, roi_bottom_right, (0, 255, 0), 2)
    cv2.putText(frame, f"Density: {density:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Congestion: {congestion_level}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Traffic Congestion Analysis", frame)

    # 11. Exit on Key Press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 640x640 3 persons, 7 cars, 2 motorcycles, 3 buss, 4 trucks, 26.5ms
Speed: 5.0ms preprocess, 26.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 7 cars, 2 motorcycles, 3 buss, 5 trucks, 25.0ms
Speed: 6.0ms preprocess, 25.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 7 cars, 3 motorcycles, 3 buss, 4 trucks, 25.7ms
Speed: 4.0ms preprocess, 25.7ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 7 cars, 1 motorcycle, 3 buss, 5 trucks, 25.0ms
Speed: 5.0ms preprocess, 25.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 7 cars, 2 motorcycles, 1 bus, 6 trucks, 25.0ms
Speed: 4.0ms preprocess, 25.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 8 cars, 1 motorcycle, 1 bus, 6 trucks, 16.1ms
Speed: 3.6ms preprocess, 16.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 64

KeyboardInterrupt: 

In [7]:
import cv2
import torch
from ultralytics import YOLO

# 1. Load Pre-trained YOLOv8 Model
model = YOLO("yolov8n.pt")  # Load a small YOLOv8 model (e.g., 'yolov8n.pt', 'yolov8s.pt', etc.)

# 2. Define Region of Interest (ROI)
#   - Adjust the coordinates based on your camera view and the road section you want to analyze
roi_top_left = (100, 100)
roi_bottom_right = (400, 300)

# 3. Load and Process Video Frame
video_path = r'C:\Users\PREDATOR\Downloads\congestion classifier\Untitled video - Made with Clipchamp.mp4' # Replace with your video file path
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # 4. Extract ROI from the Frame
    roi = frame[roi_top_left[1]:roi_bottom_right[1], roi_top_left[0]:roi_bottom_right[0]]

    # 5. Run YOLOv8 on the ROI
    results = model(roi)

    # 6. Count Vehicles
    vehicle_count = len(results)

    # 7. Calculate Density (Assume ROI area is 100 square units for this example)
    density = vehicle_count / 100

    # 8. Define Congestion Thresholds (Adjust as needed)
    high_congestion_threshold = 0.2  # Example threshold
    moderate_congestion_threshold = 0.1  # Example threshold

    # 9. Classify Congestion Level
    if density >= high_congestion_threshold:
        congestion_level = "High Congestion"
    elif density >= moderate_congestion_threshold:
        congestion_level = "Moderate Congestion"
    else:
        congestion_level = "Low Congestion"

    # 10. Display Results
    cv2.rectangle(frame, roi_top_left, roi_bottom_right, (0, 255, 0), 2)
    cv2.putText(frame, f"Density: {density:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Congestion: {congestion_level}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Traffic Congestion Analysis", frame)

    # 11. Exit on Key Press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 448x640 (no detections), 30.5ms
Speed: 4.0ms preprocess, 30.5ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 15.9ms
Speed: 3.0ms preprocess, 15.9ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 15.1ms
Speed: 2.1ms preprocess, 15.1ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 15.9ms
Speed: 2.0ms preprocess, 15.9ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 16.0ms
Speed: 2.1ms preprocess, 16.0ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 15.7ms
Speed: 2.1ms preprocess, 15.7ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 15.5ms
Speed: 2.0ms preprocess, 15.5ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 13.0ms
Speed: 2.5ms preprocess, 13.0ms i

KeyboardInterrupt: 